# DATA20001 Deep Learning - Group Project
## Image project

- Testing [resnext101_32x8d](https://pytorch.org/docs/stable/torchvision/models.html#id27) implementation of pretrained transfer learning models
- Data preprocessing includes data augmentation, shuffle and split train=17k, val=1.5k, test=1.5k
- Data augmentation IS applied
- No data balancing applied



In [1]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

## Import dependencies and select correct device

In [2]:
# import dependencies
import train_model
import eval_model
import utils
from image_dataset import ImageDataset
from data_augmentation import DataAugmentation

import time
import torch
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [3]:
device = None
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda:0")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using CPU


## Define datasets

- shuffle
- train: 17k
- valid: 1.5k
- test: 1.5k

In [4]:
mapping_file = "file_to_labels_table.csv"
df = pd.read_csv(mapping_file)
train_idx, val_idx = utils.get_train_val_indexes(df, 0.85, shuffle=True)
val_test_split_idx = int(val_idx.shape[0]*.5)
test_idx = val_idx[:val_test_split_idx]
val_idx = val_idx[val_test_split_idx:]

dataAugmentation = DataAugmentation()
trainDataset = ImageDataset(train_idx, dataAugmentation=dataAugmentation)
valDataset = ImageDataset(val_idx)
testDataset = ImageDataset(test_idx)

print("Saving test dataset temporarily so test can be performed after restarting kernel.")
utils.save_pickle(testDataset, "tmp_test_data_exp_transfer_learning_5.pkl")
print("# Train:", len(trainDataset))
print("# Valid:", len(valDataset))
print("# Test:", len(testDataset))

Saving test dataset temporarily so test can be performed after restarting kernel.
# Train: 17000
# Valid: 1500
# Test: 1500


## Define the model architecture

The initial transfer learning implementation

In [5]:
torch.cuda.empty_cache()

preTrainedModel = models.resnext101_32x8d(pretrained=True)

for param in preTrainedModel.parameters():
    param.requires_grad = False

num_ftrs = preTrainedModel.fc.in_features
num_classes = 14

preTrainedModel.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, num_classes),
    torch.nn.Sigmoid()
)

model = preTrainedModel.to(device)

## Define test case

Hyperparam search could be defined here. No hyperparam searches in this notebook as purpose of this test is to test the effect of data preprocessing technique on initial transfer learning model architecture and see how longer run on epochs affect loss.

**Test objective**:

- Testing [resnext101_32x8d](https://pytorch.org/docs/stable/torchvision/models.html#id27) implementation of pretrained transfer learning model
- Data preprocessing includes 
  - data augmentation, 
  - shuffle 
  - split train=17k, val=1.5k, test=1.5k


**Training results**:

- Best validation results:
 - Loss: 0.1113 Accuracy: 0.9619
 - Precision: 0.7223 Recall: 0.7258
 - F1: 0.7241 Duration: 2m 3s
- Testing set results.
 - Accuracy 0.963
 - Precision 0.745
 - Recall 0.742
 - F1-score 0.743

In [6]:
n_epochs = 6

model, logs = train_model.train_model(
    model,
    trainDataset,
    valDataset,
    device,
    numberOfEpochs=n_epochs,
    returnLogs=True
)

Epoch 0/6
----------
Validating...

Progress: 100%

Loss: 0.6959 Accuracy: 0.5098
Precision: 0.0681 Recall: 0.4414
F1: 0.1181 Duration: 3m 7s

Epoch 1/6
----------
Training...

Progress: 100%

Loss: 0.1407 Accuracy: 0.9500
Precision: 0.7217 Recall: 0.5002
F1: 0.5909 Duration: 46m 4s

Validating...

Progress: 100%

Loss: 0.1209 Accuracy: 0.9581
Precision: 0.7650 Recall: 0.6297
F1: 0.6908 Duration: 4m 31s

Epoch 2/6
----------
Training...

Progress: 100%

Loss: 0.1279 Accuracy: 0.9547
Precision: 0.7373 Recall: 0.5772
F1: 0.6475 Duration: 44m 44s

Validating...

Progress: 100%

Loss: 0.1094 Accuracy: 0.9613
Precision: 0.7567 Recall: 0.7072
F1: 0.7311 Duration: 2m 52s

Epoch 3/6
----------
Training...

Progress: 100%

Loss: 0.1283 Accuracy: 0.9553
Precision: 0.7424 Recall: 0.5830
F1: 0.6531 Duration: 43m 12s

Validating...

Progress: 100%

Loss: 0.1204 Accuracy: 0.9585
Precision: 0.7102 Recall: 0.7457
F1: 0.7275 Duration: 3m 4s

Epoch 4/6
----------
Training...

Progress: 100%

Loss: 0.126

## Save model and logs

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [7]:
pd.DataFrame(logs["val"]).to_csv("exp_transfer_learning_5_logs_val.csv", index=False)
pd.DataFrame(logs["train"]).to_csv("exp_transfer_learning_5_logs_train.csv", index=False)
torch.save(model, 'exp_transfer_learning_5_model.pkl')  # saves model with architecture

## Test model

- Load model back from state_dict and perform test on testing set
- Load test-set
- Load training time logs of performance

In [8]:
model = torch.load("exp_transfer_learning_5_model.pkl")
testDataset = utils.load_pickle("tmp_test_data_exp_transfer_learning_5.pkl")
logsTable = pd.read_csv("exp_transfer_learning_5_logs_val.csv")
y_hats, y_trues = eval_model.test_model(model, testDataset, device)

### Validation set performance during training

In [9]:
logsTable

,duration,loss,accuracy,precision,recall,f1,true positives,true negatives,false positives,false negatives
0,187.343715,0.695931,0.509810,0.068144,0.441384,0.118060,689,10017,9422,872
1,271.170860,0.120881,0.958095,0.764981,0.629725,0.690794,983,19137,302,578
2,171.535487,0.109356,0.961333,0.756683,0.707239,0.731126,1104,19084,355,457
3,183.650126,0.120381,0.958476,0.710189,0.745676,0.727500,1164,18964,475,397
4,180.207290,0.125977,0.957095,0.739478,0.652787,0.693433,1019,19080,359,542
5,188.540844,0.137415,0.954476,0.713178,0.648302,0.679195,1012,19032,407,549
6,186.874344,0.118031,0.960905,0.767341,0.680333,0.721222,1062,19117,322,499


### Testing set performance

In [10]:
print("Testing set results.")
print(f" Accuracy\t{round(eval_model.get_metric(y_trues, y_hats, 'accuracy'), 3)}")
print(f" Precision\t{round(eval_model.get_metric(y_trues, y_hats, 'precision'), 3)}")
print(f" Recall \t{round(eval_model.get_metric(y_trues, y_hats, 'recall'), 3)}")
print(f" F1-score\t{round(eval_model.get_metric(y_trues, y_hats, 'f1'), 3)}")

Testing set results.
 Accuracy	0.959
 Precision	0.716
 Recall 	0.705
 F1-score	0.71


### Prediction analysis

- How many positives and negatives has been predicted.
  - Tuples like: **(label, count)**
- Binary confusion matrix for each label

In [11]:
list(zip(*np.unique(y_hats, return_counts=True)))

[(0, 19493), (1, 1507)]

In [11]:
from sklearn.metrics import multilabel_confusion_matrix

label_names = ["clouds", "male", "bird", "dog", "river", "portrait", "baby", "night", "people", "female", "sea", "tree", "car", "flower"]
results = multilabel_confusion_matrix(y_trues, y_hats)

In [12]:
print("\tRecall\tPrecision")
for i, conf_mtx in enumerate(results):
    tp = conf_mtx[1, 1]
    tn = conf_mtx[0, 0]
    fp = conf_mtx[0, 1]
    fn = conf_mtx[1, 0]
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    print(f"{label_names[i][:6]}\t{round(recall, 3)}\t{round(precision, 3)}")

	Recall	Precision
clouds	0.649	0.568
male	0.505	0.635
bird	0.741	0.87
dog	0.925	0.507
river	0.0	nan
portra	0.74	0.778
baby	0.0	nan
night	0.649	0.324
people	0.824	0.867
female	0.658	0.69
sea	0.6	0.571
tree	0.765	0.553
car	0.722	0.867
flower	0.684	0.722


/home/kasperireinikainen/env3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  


## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [12]:
np.savetxt('results.txt', y, fmt='%d')

NameError: name 'y' is not defined